In [ ]:
# def compute_kernel_matrix(X):
#     n_samples = X.shape[0]
#     K = np.zeros((n_samples, n_samples))
#     # for i in range(n_samples):
#     #     for j in range(n_samples):
#     #         K[i, j] = np.dot(X[i], X[j])
#     K = np.matmul(X,X.T)
#     return K

def svm_dual(X, y, C=1.0):
    from scipy.optimize import minimize
    n_samples = X.shape[0]
    K = np.matmul(X,X.T) #compute_kernel_matrix
    def objective(alpha):
      alpha = np.reshape(alpha,(len(alpha),1))
      yy = np.reshape(y,(len(alpha),1))
      return 0.5 * alpha.T @ (K*(yy@yy.T)) @ alpha - np.ones((1,len(alpha))) @ alpha
      # objective_value1=0
      # for i in range(len(y)):
      #   for j in range(len(y)):
      #     objective_value1=objective_value1+alpha[i]*alpha[j]*y[i]*y[j]*np.dot(X[i,:],X[j,:].T)
      # return (+1/2)*objective_value1-np.sum(alpha)

    # Quadratic programming
    # objective = lambda alpha: -np.sum(alpha) + 0.5 * np.dot(alpha, np.dot(np.dot(np.diag(y), K), np.diag(y)))  # Objective function
    constraints = ({'type': 'eq', 'fun': lambda alpha: np.dot(alpha, y), 'jac': lambda alpha: y})  # Equality constraint
    bounds = [(0, C) for _ in range(n_samples)]  # Bounds for alpha (0 <= alpha <= C)
    alpha_initial_guess = np.random.rand(n_samples)  # Initial guess for alpha
    alpha_solution = minimize(objective, alpha_initial_guess, bounds=bounds, constraints=constraints)
    alpha = alpha_solution.x
    return alpha

def compute_decision_boundary(X, y, alpha):
    # Find support vectors (non-zero alphas)
    sv_idx = np.where(alpha > 1e-6)[0]
    sv_alpha = alpha[sv_idx]
    sv_X = X[sv_idx]
    sv_y = y[sv_idx]

    # Calculate weights (w) and bias (b)
    w = np.dot(sv_alpha * sv_y, sv_X)
    b = np.mean(sv_y - np.dot(sv_X, w))

    return w, b

def ehsan_svm_train(X, yy, C=1.0):
  import numpy as np
  unique_lables=[]
  for lable in yy:
    # check if exists in unique_list or not
    if lable not in unique_lables:
      unique_lables.append(lable)
  if(len(unique_lables)!=2):raise Exception("Sorry, wrong dataset")
  y=np.ones(np.shape(yy))
  y[np.where(yy<max(unique_lables))[0]]=-1
  n_samples, n_features = X.shape
  if(n_samples!=len(y)):raise Exception("Sorry, wrong dataset")
  alpha=svm_dual(X, y,C)
  w, b = compute_decision_boundary(X, y, alpha)
  w_new=np.reshape(w,(1,2))
  TP=0
  TN=0
  for i in range(n_samples):
    if(np.dot(w_new,X[i,:].T)+b>0 and y[i]>0):
      TP=TP+1
    if(np.dot(w_new,X[i,:].T)+b<0 and y[i]<0):
      TN=TN+1
  acc_train=(TP+TN)/len(y)*1.0
  return w,b,acc_train

def ehsan_svm_test(w,b,X,yy):
  import numpy as np
  unique_lables=[]
  for lable in yy:
    # check if exists in unique_list or not
    if lable not in unique_lables:
      unique_lables.append(lable)
  if(len(unique_lables)!=2):raise Exception("Sorry, wrong dataset")
  y=np.ones(np.shape(yy))
  y[np.where(yy<max(unique_lables))[0]]=-1
  n_samples, n_features = X.shape
  if(n_samples!=len(y)):raise Exception("Sorry, wrong dataset")
  w_new=np.reshape(w,(1,2))
  TP=0
  TN=0
  for i in range(n_samples):
    if(np.dot(w_new,X[i,:].T)+b>0 and y[i]>0):
      TP=TP+1
    if(np.dot(w_new,X[i,:].T)+b<0 and y[i]<0):
      TN=TN+1
  acc_test=(TP+TN)/len(y)*1.0
  return acc_test



In [ ]:
import numpy as np
import matplotlib.pyplot as plt

N_samples = 1000
data_dimension = 2
X = np.random.rand(N_samples, data_dimension)
X[0:int(np.shape(X)[0]/2),0]=X[0:int(np.shape(X)[0]/2),0]+0.65
X[0:int(np.shape(X)[0]/2),1]=X[0:int(np.shape(X)[0]/2),1]-0.65
y=np.zeros((N_samples,))

# np.random.seed(0)
# X = np.random.randn(100, 2)
# y = np.random.choice([-1, 1], size=100)

for i in range(0,np.shape(X)[0]):
  if i<np.shape(X)[0]/2:
    y[i]=-1
  else:
    y[i]=+1
a=np.random.permutation(N_samples)
X=X[a,:]
y=y[a]

X_train=X[0:int(np.shape(X)[0]*0.5),:]
X_test=X[int(np.shape(X)[0]*0.5):np.shape(X)[0],:]

y_train=y[0:int(np.shape(X)[0]*0.5)]
y_test=y[int(np.shape(X)[0]*0.5):np.shape(X)[0]]

del(X,y)

w,b,acc_train=ehsan_svm_train(X_train,y_train,C=0.01)
print("accuracy of train:",acc_train)
print("Weights (w):", w)
print(np.shape(w))
print("Bias (b):", b)

acc_test=ehsan_svm_test(w,b,X_test,y_test)
print("accuracy of test:",acc_test)

plt.figure()
a0=np.where((y_train>0)*1)
b0=np.where((y_train<0)*1)
plt.scatter(X_train[a0[0],0], X_train[a0[0],1])
plt.scatter(X_train[b0[0],0], X_train[b0[0],1])
plt.plot(X_train[:,0],-1/(w[1])* (b+w[0]*X_train[:,0]))
plt.title('train')


plt.figure()
a0=np.where((y_test>0)*1)
b0=np.where((y_test<0)*1)
plt.scatter(X_test[a0[0],0], X_test[a0[0],1])
plt.scatter(X_test[b0[0],0], X_test[b0[0],1])
plt.plot(X_test[:,0],-1/(w[1])* (b+w[0]*X_test[:,0]))
plt.title('test')

In [ ]:
# comparing sckit-learn SVM and MY_SVM

import numpy as np
import matplotlib.pyplot as plt
from sklearn import svm
import time

clf = svm.SVC()

N_samples = 200
data_dimension = 2
N_iter=5
acc_train=np.zeros((N_iter))
acc_test=np.zeros((N_iter))
time_train=np.zeros((N_iter))
scikit_acc_train=np.zeros((N_iter))
scikit_acc_test=np.zeros((N_iter))
scikit_time_train=np.zeros((N_iter))

for jj in range(N_iter):
  X = np.random.rand(N_samples, data_dimension)
  X[0:int(np.shape(X)[0]/2),0]=X[0:int(np.shape(X)[0]/2),0]+0.65
  X[0:int(np.shape(X)[0]/2),1]=X[0:int(np.shape(X)[0]/2),1]-0.65
  y=np.zeros((N_samples,))

  # np.random.seed(0)
  # X = np.random.randn(100, 2)
  # y = np.random.choice([-1, 1], size=100)

  for i in range(0,np.shape(X)[0]):
    if i<np.shape(X)[0]/2:
      y[i]=-1
    else:
      y[i]=+1
  a=np.random.permutation(N_samples)
  X=X[a,:]
  y=y[a]

  X_train=X[0:int(np.shape(X)[0]*0.5),:]
  X_test=X[int(np.shape(X)[0]*0.5):np.shape(X)[0],:]

  y_train=y[0:int(np.shape(X)[0]*0.5)]
  y_test=y[int(np.shape(X)[0]*0.5):np.shape(X)[0]]

  del(X,y)

  tic = time.time()
  w,b,acc_train[jj]=ehsan_svm_train(X_train,y_train,C=0.01)
  time_train[jj] = time.time()-tic
  acc_test[jj]=ehsan_svm_test(w,b,X_test,y_test)


  tic = time.time()
  clf.fit(X_train, y_train)
  scikit_time_train[jj] = time.time()-tic
  yy=clf.predict(X_train)
  scikit_acc_train[jj]=sum(yy==y_train)/len(y_train)
  yy=clf.predict(X_test)
  scikit_acc_test[jj]=sum(yy==y_test)/len(y_test)
print('acc_train',np.mean(acc_train))
print('acc_test',np.mean(acc_test))
print('time_train',np.mean(time_train))
print('scikit_acc_train',np.mean(scikit_acc_train))
print('scikit_acc_test',np.mean(scikit_acc_test))
print('scikit_time_train',np.mean(scikit_time_train))





acc_train 0.952
acc_test 0.9400000000000001
time_train 0.15453600883483887
scikit_acc_train 0.962
scikit_acc_test 0.942
scikit_time_train 0.0020351409912109375
